### __0. prepare__






* http://www.datamarket.kr/xe/index.php?mid=board_pdzw77&document_srl=71220&m=1&listStyle=viewer&page=1
* https://github.com/tobigs-reco-conf-1314/Tommelier
* https://github.com/tobigs-reco-conf-1314/Tommelier/blob/main/Results/NeuralMF_main.ipynb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
import gc
import glob
import joblib
from tqdm import tqdm

import ast
import numpy as np
import pandas as pd

import keras
import tensorflow as tf

In [ ]:
# 데이터 불러오기
df = pd.read_csv('/content/drive/Shareddrives/컨퍼런스/preprocess_total.csv')

# 리뷰 10개 이상 가구 수
df2 = df['item_name'].value_counts().reset_index()
print('총 가구 수:', len(df2))

# 리뷰 3개 이상 유저 수
df3 = df['user'].value_counts().reset_index()
print('총 유저 수:', df['user'].nunique())
print('3개 이상 유저 수:', len(df3.query('user >= 3')))

# 최종 사용할 데이터셋
my_user = df3.query('user >= 3')['index'].tolist()
isin_con = df['user'].isin(my_user)
df = df.loc[isin_con].reset_index().iloc[:, 1:]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


총 가구 수: 4368
총 유저 수: 660325
3개 이상 유저 수: 139610


In [ ]:
# rating_per_user 변수 생성
# 유저별 평균 평점 
df4 = df.groupby('user')['total_score'].mean().reset_index()
df = pd.merge(left=df, right=df4, on='user', how='left')
df = df.rename(columns={'total_score_x': 'total_score', 'total_score_y': 'rating_per_user'})

# like 변수 생성 
# 가구에 대한 유저의 평점 > 해당 유저의 평균 평점 -> 1
# 가구에 대한 유저의 평점 < 해당 유저의 평균 평점 -> 0
def mylike(df):
    if df['total_score'] >= df['rating_per_user']:
        return 1
    else:
        return 0

df['like'] = df.apply(mylike, axis=1)

# like 분포 확인
df['like'].value_counts()

1    405358
0    167596
Name: like, dtype: int64

### __1. preprocess__

In [ ]:
# *train/test 분리 
print(len(df))
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
train = df[:458363] # 80%
test = df[458363:]

#데이터프레임 타입 shuffled X
# tf.random.set_seed(42)
# shuffled = df.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
# train = shuffled.take(80_000)
# test = shuffled.skip(80_000).take(20_000)

572954


In [ ]:
train = train[['user', 'item_name', 'like']]
test = test[['user', 'item_name', 'like']]

In [ ]:
user_ids = train['user'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

item_ids = train['item_name'].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}

In [ ]:
add_train = pd.DataFrame(zip(train['user'].map(user2user_encoded), train["item_name"].map(item2item_encoded), train['like']),
             columns = ['user', 'item_name', 'like'])

add_test = pd.DataFrame(zip(test['user'].map(user2user_encoded), test["item_name"].map(item2item_encoded), test['like']),
             columns = ['user', 'item_name', 'like'])

In [ ]:
X = add_train[['user', 'item_name']].values
y = add_train['like'].values

### __2. Model__

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 42) # stratify=y 추가 

In [ ]:
!git clone https://github.com/tobigs-reco-conf-1314/Tommelier

Cloning into 'Tommelier'...
remote: Enumerating objects: 797, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 797 (delta 101), reused 75 (delta 25), pack-reused 605
Receiving objects: 100% (797/797), 6.19 MiB | 23.22 MiB/s, done.
Resolving deltas: 100% (399/399), done.


In [ ]:
#sys.path.append(os.path.join('..', 'Models'))
%cd /content/Tommelier/Models

/content/Tommelier/Models


In [ ]:
num_users = len(user2user_encoded)
num_itmes = len(item_encoded2item)
EMBEDDING_SIZE = 256 #50

In [ ]:
import NeuralMF
from tensorflow import keras

model = NeuralMF.RecommenderNet(num_users, num_itmes, EMBEDDING_SIZE)

model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(), 
    optimizer = keras.optimizers.Adam(lr = 0.005),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(
            name='binary_accuracy', dtype = None, threshold = 0.5)
    ])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


### __3. Train__

In [ ]:
callbacks_list = [
                  keras.callbacks.EarlyStopping(
                      monitor = 'binary_accuracy',
                      patience = 5
                      )]

In [ ]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 128,
    epochs = 10,
    verbose = 1,
    validation_data = (x_val, y_val),
)

Epoch 1/10
2865/2865 [==============================] - 990s 345ms/step - loss: 3.5809 - binary_accuracy: 0.5544 - val_loss: 7.5955 - val_binary_accuracy: 0.5233
Epoch 2/10
2865/2865 [==============================] - 1029s 359ms/step - loss: 21.3829 - binary_accuracy: 0.5416 - val_loss: 11.8688 - val_binary_accuracy: 0.5169
Epoch 3/10
2865/2865 [==============================] - 1023s 357ms/step - loss: 36.7987 - binary_accuracy: 0.5002 - val_loss: 16.1196 - val_binary_accuracy: 0.6151
Epoch 4/10
2865/2865 [==============================] - 1030s 360ms/step - loss: 40.3560 - binary_accuracy: 0.6282 - val_loss: 29.9640 - val_binary_accuracy: 0.3741
Epoch 5/10
2865/2865 [==============================] - 1041s 363ms/step - loss: 62.2157 - binary_accuracy: 0.5137 - val_loss: 21.8433 - val_binary_accuracy: 0.6287
Epoch 6/10
2865/2865 [==============================] - 1023s 357ms/step - loss: 46.9837 - binary_accuracy: 0.5763 - val_loss: 21.8140 - val_binary_accuracy: 0.5386
Epoch 7/10
28

In [ ]:
# 모델 전체 저장
model.save('NMF_model_epoch10')
new_model = tf.keras.models.load_model('NMF_weights_epoch10')

test_loss, test_acc = new_model.evaluate(x_val,  y_val, verbose=2)

INFO:tensorflow:Assets written to: NMF_model_epoch10/assets


OSError: ignored

In [ ]:
# 모델 가중치 저장 
model.save_weights('NMF_weights_epoch10')

# 가중치는 모델 architecture 다시 지정 후 사용 
# new_model= tf.keras.models.Sequential([ 
#     tf.keras.layers.Dense(32, input_dim=x.shape[1]),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
# ])

new_model.load_weights('NMF_weights_epoch10')
test_loss, test_acc = new_model.evaluate(x,  y, verbose=2)